# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [17]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [18]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

C:\Users\arnaldo\Documents\jupyter-documents\udacity-nanodegree-de\project2


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [19]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [20]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

411


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [21]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()
# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [22]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_events 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [25]:
try:
    session.set_keyspace('music_events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [26]:
query = "CREATE TABLE IF NOT EXISTS session_events "
query = query + "(sessionId int, itemInSession int, userId int, firstName text, lastName text, \
                    level text, gender text, location text, song text, artist text, length double, \
                    PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [27]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO session_events (sessionId, itemInSession, userId, firstName, lastName, level, gender, \
                                    location, song, artist, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[1], line[4], 
                                line[6], line[2], line[7], line[9], line[0], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [28]:
query = "SELECT artist, song, length FROM session_events WHERE sessionId=128 AND itemInSession=16"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.length)

Erykah Badu Back In The Day 286.9024


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [29]:
query = "CREATE TABLE IF NOT EXISTS user_events "
query = query + "(sessionId int, itemInSession int, userId int, firstName text, lastName text, \
                    level text, gender text, location text, song text, artist text, length double, \
                    PRIMARY KEY (userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)   

In [30]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_events (sessionId, itemInSession, userId, firstName, lastName, level, gender, \
                                    location, song, artist, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[1], line[4], 
                                line[6], line[2], line[7], line[9], line[0], float(line[5])))

In [31]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "SELECT artist, song, firstname, lastname FROM user_events WHERE userId=25 AND sessionId=128"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Deerhunter Weird Era Jayden Graves
Daughtry No Surprise Jayden Graves
Vinylshakerz Club Tropicana Jayden Graves
Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile) Jayden Graves
Son House Depot Blues Jayden Graves
Jason Mraz & Colbie Caillat Lucky (Album Version) Jayden Graves
The Romantics Talking In Your Sleep Jayden Graves
The Postal Service Such Great Heights Jayden Graves
Tweet Always Will (LP Version) Jayden Graves
Gang Starr/Inspectah Deck Above The Clouds (Edited) Jayden Graves
Kanye West Coldest Winter Jayden Graves
Shunza Nan Ni Wan Jayden Graves
Erykah Badu Back In The Day Jayden Graves
Asia 2001 Epilogue Jayden Graves
The Casualties Fight For Your Life Jayden Graves
Alliance Ethnik ReprÃÂ©sente Jayden Graves
Hey Monday Run_ Don't Walk Jayden Graves
Silverchair The Door Jayden Graves
El DeBarge Love Always Jayden Graves
Evanescence Bring Me To Life Jayden Graves
Ludovico Einaudi The Snow 

In [32]:
query = "CREATE TABLE IF NOT EXISTS song_events "
query = query + "(sessionId int, itemInSession int, userId int, firstName text, lastName text, \
                    level text, gender text, location text, song text, artist text, length double, \
                    PRIMARY KEY (song, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)   

In [33]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_events (sessionId, itemInSession, userId, firstName, lastName, level, gender, \
                                    location, song, artist, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[1], line[4], 
                                line[6], line[2], line[7], line[9], line[0], float(line[5])))

In [34]:
#Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "SELECT firstname, lastname FROM song_events WHERE song='Talking In Your Sleep'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jayden Graves


### Drop the tables before closing out the sessions

In [35]:
query = "DROP TABLE IF EXISTS session_events "
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE IF EXISTS user_events "
try:
    session.execute(query)
except Exception as e:
    print(e)
query = "DROP TABLE IF EXISTS song_events "
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [36]:
session.shutdown()
cluster.shutdown()